Project 5 - Vehicle Detection - Report. Vilas Chitrakaran. March 2017

# Executive Summary
The objective is to develop a software pipeline to identify and track vehicles in a video stream obtained from a front-facing camera mounted on the dashboard of a car. A linear support vector machine (SVM) is trained to identify vehicles based on histogram-of-oriented-gradients (HOG) features extracted from 'car' and 'non-car' image sets. The trained SVM is then applied on the video stream to search and identify vehicles. The robustness of the identification pipeline is improved by temporal filtering techniques to discard false positives. 

The following video demonstrates the final result obtained from my implementation.

__TODO__: Link to final video

# Processing Pipeline Development

The vehicle is instrumented with a front-facing video camera that provides an RGB video stream of the road ahead. 

## Feature Extraction - Histogram of Oriented Gradients (HOG)
Why use HOG and not something else?

Explain how (and identify where in your code) you extracted HOG features from the training images. Explain how you settled on your final choice of HOG parameters.
## Classifier Training - Support Vector Machine

Why use SVM and not something else?

Describe how (and identify where in your code) you trained a classifier using your selected HOG features (and color features if you used them).

## Vehicle Detection - Sliding Window Search
Describe how (and identify where in your code) you implemented a sliding window search. How did you decide what scales to search and how much to overlap windows?
Show some examples of test images to demonstrate how your pipeline is working. How did you optimize the performance of your classifier?

## Processing Pipeline
Summarise the steps in the processing pipeline. 

Describe how (and identify where in your code) you implemented some kind of filter for false positives and some method for combining overlapping bounding boxes.

Provide a link to your final video output. Your pipeline should perform reasonably well on the entire project video (somewhat wobbly or unstable bounding boxes are ok as long as you are identifying the vehicles most of the time with minimal false positives.)

Combine vehicle detection with lane detection

# Vehicle Detection - Project Video
All that remains now is to process the video sequence by applying the processing pipeline described in the previous section.

In [ ]:
#=====================================================================
# process the project video

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

output_video_name = 'output.mp4'
input_video = VideoFileClip("project_video.mp4")

output_video = input_video.fl_image(process_image)
%time output_video.write_videofile(output_video_name, audio=False)

# Conclusions
Briefly discuss any problems / issues you faced in your implementation of this project. Where will your pipeline likely fail? What could you do to make it more robust?